In [1]:
import os
import time
import math

label = 'cz' # 数据标签
save_file = 'data/faceimg/'
length = 0
gap_time = 1
video_type = './data/facevideo/3.mp4'  # 获取视频的方式，=0为默认摄像头，等于地址为视频的地址
recognize_type = 0  # 分类方式，0:对CNN产生的特征进行KNN，1:不经过CNN直接KNN,2:用lightCnn直接分类
dis_type = 'cosine'  # 距离函数
kernel_type = 0  # 核函数 0：线性核 1：RBF核


# Capture webcam image
import numpy as np
import cv2
from light_cnn_test import shift_img_with_eyes
from read_face_data import read_face_data
from knn_10_20 import get_Kernels,KNN#get_d2
from hyperparams import Hyperparams
hp = Hyperparams()

#  读取名字列表
name = ['error']
name.extend(read_face_data('data/faceimg/', 0))
name.append('error')
print(name)


['error', 'cz', 'zh', 'error']


In [2]:
from test_light_cnn_in_numpy import get_feature, get_net

net = get_net("data/lightCNN_pretrain0.npy")
print("Finish")

Loading network...
Finish


In [9]:
tem_trains,labels = read_face_data('data/faceimg/',2)
trains = []
for num, train in enumerate(tem_trains):
    tem = train[np.newaxis, :, :, :]
    tem = get_feature(net, tem).reshape([1,-1])
    trains.append(tem)
trains = np.concatenate(trains, axis=0)
print(trains.shape)
print(labels)

data/faceimg/cz/0.jpg
data/faceimg/cz/1.jpg
data/faceimg/cz/2.jpg
data/faceimg/cz/3.jpg
data/faceimg/cz/4.jpg
data/faceimg/zh/0.jpg
data/faceimg/zh/1.jpg
data/faceimg/zh/2.jpg
data/faceimg/zh/3.jpg
data/faceimg/zh/4.jpg
(10, 1024)
[1 1 1 1 1 2 2 2 2 2]


In [10]:
face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')

In [11]:
aa = {'asdsa':'ahello'}
np.save("data//tttt.npy",aa)

In [12]:
from pynq import Overlay
overlay = Overlay('pwm_1.bit')
from motor import *
from steer import *
motor = Motor(overlay)
steer = Steer(overlay)
motor.set_speed(0, 0)
#steer.setangle(0,0)

In [13]:
from pynq.lib.video import *
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [14]:
# initialize camera from OpenCV
import cv2

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))
print(cv2.CAP_PROP_FRAME_WIDTH)

Capture device is open: True
3


In [15]:
steer.set_angle(0, 0)

In [17]:
steer.set_angle(0, 0)
import math
import time

TURN_LEFT = 0
TURN_RIGHT = 1

length = 0
cheak_times = 0
steer.set_angle(0, 0)
steer_flag = 0
if_have_face = False
patrol_w = 5
patrol_h = 8
runspeed = [0,0]
no_faces_times = 0  # 记录当前连续未出现目标头像的循环数
while(True) :
    while videoIn.isOpened() :
        if_have_face = False
        ret, Vshow = videoIn.read()  # 重复获取以保证获取的是最新的数据
        
        
        
        np_frame = Vshow

        cheak_times += 1
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        print(faces)
        if type(faces) != 'list' and no_faces_times > 20:
            save_file_name = save_file + 'pre_result/' + str(cheak_times) + "_000"  + ".jpg"
            cv2.imwrite(save_file_name, Vshow)
            print("patrol")
            if steer_flag == TURN_LEFT :
                if steer.horAngle > -75:
                    steer.adj_angle(-patrol_w,0)
                else:
                    if steer.verAngle - patrol_h*2 < -85:
                        steer.set_angle(-80,0)
                    else:
                        steer.adj_angle(0,-patrol_h*2)
                    steer_flag = TURN_RIGHT
            elif steer_flag == TURN_RIGHT :
                if steer.horAngle < 75:
                    steer.adj_angle(patrol_w,0)
                else:
                    if steer.verAngle + patrol_h*2 > 45:
                        steer.set_angle(86,0)
                    else:
                        steer.adj_angle(0,patrol_h)
                    steer_flag = TURN_LEFT
            time.sleep(0.800)
            print("v:%f h:%f"%(steer.verAngle, steer.horAngle))
                    
        print(steer_flag)
        for (x,y,w,h) in faces:
            steer_flag = 0
            if_have_face = True
            #cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
            #roi_gray = gray[y:y+h, x:x+w]
            #roi_color = np_frame[y:y+h, x:x+w]

            #eyes = eye_cascade.detectMultiScale(roi_gray)
            #for (ex,ey,ew,eh) in eyes:
                #cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            face = Vshow[y:y+h,x:x+w]
            face = shift_img_with_eyes(face)
            
            # face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)  # [3]
            # face = np.mean(face, 2)
            # now_face = face.reshape((-1,now_face.shape[0]*now_face.shape[1]))

            # now_face = np.transpose(face,(2,0,1))
            # now_face = now_face.reshape((-1,1,now_face.shape[1]*now_face.shape[2]))
            # now_face = now_face.astype(np.float32)
            now_face = face
            print("first",str(now_face.shape))
            now_face = now_face[np.newaxis, :, :, :]
            now_face = np.transpose(now_face, (0, 3, 1, 2))
            now_face = get_feature(net, now_face).reshape([1,-1])
            print("second",str(now_face.shape))
            
            print(now_face.shape)
            predicted = KNN(trains,labels,now_face)
            print("predicted:" + str(predicted) + name[predicted])
            
            text = name[predicted]
            cv2.putText(Vshow, text, (x, y),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))
            cv2.rectangle(Vshow, (x,y),(x+w,y+h),(255,0,0),2)
            save_file_name = save_file + 'pre_result/' + str(length) + "_" + str(predicted)  + ".jpg"
            length += 1
            print(save_file_name)
            cv2.imwrite(save_file_name, Vshow)
            
            if predicted == 1 :
                faceSizePct = w/frame_in_w
                widthOffset = frame_in_w/2 - (x + w/2)
                heightOffset = frame_in_w/2 - (y + w/2)
                deltaThetaX = math.degrees (atan(2 * widthOffset * tan(0.675) / L))
                deltaThetaY = math.degrees (atan(2 * heightOffset * tan(0.675) / L))
                # deltaThetaX = widthOffset / frame_in_w * 68
                # deltaThetaY = - heightOffset / frame_in_w * 68

                if runspeed[0] == 0 and faceSizePct - 0.6 < 0 :
                    baseSpeed = (abs(faceSizePct-0.6) ** 2) * 250
                    runspeed[0] = runspeed[1] = baseSpeed;
                    print(baseSpeed)
                    if widthOffset > 0:
                        runspeed[1] += min(deltaThetaX / 9 ,10)
                    else:
                        runspeed[0] += min(deltaThetaX / 9,10)
                    #runspeed[0] = min(baseSpeed - (frame_in_w/2 - (x + w/2)) * 0.2,100)
                    #runspeed[1] = min(baseSpeed + (frame_in_w/2 - (x + w/2)) * 0.2,100)
                    print(runspeed)
                #elif widthOffset <= 0.08:
                    #runspeed[0] = 0
                    #runspeed[1] = 0
                if abs(deltaThetaX) < 1.5:
                    x_angle = 0
                if abs(deltaThetaY) < 1.5:
                    y_angle = 0
                steer.adj_angle(deltaThetaX, deltaThetaY)
                print("adj angle: %d,%d speed: %d,%d"%(deltaThetaX, deltaThetaY, runspeed[0], runspeed[1]))
                no_faces_times = 0
        else:
            no_faces_times += 1
        if no_faces_times == 3:  # 连续三循环（照片）没有找到目标，就停下来
            runspeed[0] = 0
            runspeed[1] = 0
        print("runspeed: " + str(runspeed))
        motor.set_speed(runspeed[0],runspeed[1])

        if cheak_times % 15 == 0:
            save_file_name = save_file + 'cheak/' + str(cheak_times) + ".jpg"
            print(save_file_name)
            cv2.imwrite(save_file_name, Vshow)
        '''
        if if_have_face:
            for i in range(3):
                ret, Vshow = videoIn.read()
        '''
        #if length > 10:
        #    break
    videoIn.release()
    videoIn = cv2.VideoCapture(0)
    videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
    videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

    print("Capture device is open: " + str(videoIn.isOpened()))

()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
data/faceimg/cheak/15.jpg
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-5.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-10.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-15.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-20.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-25.000000
0
runspeed: [0, 0]
[[189  43 116 116]]
patrol
v:0.000000 h:-30.000000
0
first (112, 112, 3)
second (1, 1024)
(1, 1024)
predicted:2zh
data/faceimg/pre_result/0_2.jpg
runspeed: [0, 0]
()
patrol
v:0.000000 h:-35.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-40.00000

patrol
v:-40.000000 h:-65.000000
0
runspeed: [0, 0]
()
patrol
v:-40.000000 h:-70.000000
0
runspeed: [0, 0]
()
patrol
v:-40.000000 h:-75.000000
0
runspeed: [0, 0]
data/faceimg/cheak/150.jpg
()
patrol
v:-56.000000 h:-75.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-70.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-65.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-60.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-55.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-50.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-45.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-40.000000
1
runspeed: [0, 0]
()
patrol
v:-56.000000 h:-35.000000
1
runspeed: [0, 0]
[[ 55  37 331 331]]
patrol
v:-56.000000 h:-30.000000
1
first (112, 112, 3)
second (1, 1024)
(1, 1024)
predicted:2zh
data/faceimg/pre_result/6_2.jpg
runspeed: [0, 0]
[[ 67 142 255 255]]
patrol
v:-56.000000 h:-35.000000
0
first (112, 112, 3)
second (1, 1024)
(1, 1024)
predicted:2zh
data/faceimg/pre_result/7_2.jpg
run

v:-72.000000 h:-60.000000
0
runspeed: [0, 0]
()
patrol
v:-72.000000 h:-65.000000
0
runspeed: [0, 0]
()
patrol
v:-72.000000 h:-70.000000
0
runspeed: [0, 0]
()
patrol
v:-72.000000 h:-75.000000
0
runspeed: [0, 0]
()
patrol
v:0.000000 h:-80.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-75.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-70.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-65.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-60.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-55.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-50.000000
1
runspeed: [0, 0]
data/faceimg/cheak/300.jpg
()
patrol
v:0.000000 h:-45.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-40.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-35.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-30.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-25.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-20.000000
1
runspeed: [0, 0]
()
patrol
v:0.000000 h:-15.000000
1
runspeed: [0, 0]
()


KeyboardInterrupt: 

In [9]:
videoIn.release()